## Functions Definitions

In [1]:
def extract_words(text):
    '''
    Assumptions:
        - I will deal abberviaions words as separte words e.g. Mr. Ahmed as 'Mr' and 'Ahmed'
        - I will deal name with space within as seprate words e.g. Coca Cola as 'Coca' and 'Cola'
        - I am assuming that my sentences/input text can only contain " '.', ',', '?', and '!'. 
          however this concept can be extended in my code similarly as mentioned spatial characters are dealt.
    
    
    Input:
        text - A string variable contains the text to be tokenized
    returns:
        words - list of strings, contains the list of tokenized words
        
    '''
    words = text.replace('.',' . ').replace(',',' , ').replace('?', ' ? ').replace('!', ' ! ').split(' ')
        
    ept_count = 0
    for i in range(0, len(words)):
        if (words[i] == ''):
            ept_count+=1
    
    
    for i in range(0, ept_count):
        words.remove('')
    
    return words

In [2]:
def find_context(conc_list, word):
    '''
    This function returns that what was before and after the word in the list of strings
    
    inputs:
        conc_list - the list of strings
        word - the key OR the word to split list
        
    returns:
        pre - the list of strings before 'word'
        post - the list of strings after 'word'
    '''
    
    centr = 0
    
    for i in range(0, len(conc_list)):
        if (conc_list[i] == word):
            centr = i
            break
    
    pre = list(conc_list[:centr])
    post = list(conc_list[(centr+1):])
    
    return pre, post

In [3]:
def concordance(text, word, size = 3):
    '''
    input:
        text - the sentece/ text to perform algorithm
        word - the key / or the word to find
        size - the size of window to lookup, set=3 as default
    
    return:
        out - the result of concordance
        words - the list of tokenized words
    '''
    
    words = extract_words(text)
    
    match = False
    match_indexes = []
    
    for i in range(0, len(words)):
        if (words[i] == word):
            match_indexes.append(i)
            match = True
    
    # now match_indexes contains the indexes where the word is found
    
    out = []
    
    if (match == True):
        for i in range(0, len(match_indexes)):
            match = []
            if((match_indexes[i]-size) >= 0):
                pre = words[(match_indexes[i]-size):match_indexes[i]]
            else:
                pre = words[0:match_indexes[i]]
                
            for a in range(0, len(pre)):
                match.append(pre[a])
                
            match.append(words[match_indexes[i]])
            
            post = words[(match_indexes[i]+1):(match_indexes[i]+size+1)]
            
            for i in range(0, len(post)):
                match.append(post[i])
            
            out.append(match)
    else:
        out = ["word not found"]
    
    return out, words

In [4]:
def match_indexes(win_words, target_words):
    
    indexes = []
    last = 0
    for i in range(0, len(target_words)):
        for j in range(last, len(win_words)):
            if(target_words[i] == win_words[j]):
                indexes.append(j)
                last = j
                
    return indexes

In [5]:
def approximate_similar(text, word, window_size, matches_size):
    '''
    input:
        text - The sentence / text to perform algorithm
        word - The key/word to find context
        window_size - Window size for context lookup
        matches_size - Number of matches to satisfy approximation
    returns:
        out - Set containing all words with approximated context as ‘word’
    '''
    
    conc_lists, words = concordance(text, word, window_size)
    
    # print("conc_lists:\t\t" + str(conc_lists) + "\n\n")
    
    if (conc_lists != ["word not found"]):
        matches = set({})

        for a_conc_idx in range(0, len(conc_lists)):
            pre_list, post_list = find_context(conc_lists[a_conc_idx], word)

            for b_word_idx in range(0, len(words)): # iteraing all words

                for c_pre_win in range(0, window_size):
                    pre_win_start = b_word_idx
                    pre_win_end = b_word_idx + c_pre_win

                    pre_win_words = words[pre_win_start:pre_win_end]

                    pre_match_indices = match_indexes(pre_win_words, pre_list) #returns list of matched indexes
                    
                    #print()
                    
                    if (len(pre_match_indices) >= matches_size):
                        center_start = pre_match_indices.index(max(pre_match_indices)) # max index in list
                        center_start += pre_win_start

                        for d_post_win in range(0, window_size):
                            post_win_start = center_start + 1
                            post_win_end = post_win_start + d_post_win

                            post_win_words = words[post_win_start:post_win_end]

                            post_match_indicies = match_indexes(post_win_words, post_list)

                            if(len(post_match_indicies) >= matches_size):
                                center_end = post_match_indicies.index(min(post_match_indicies)) # min index in list
                                center_end += post_win_start

                                matches.update(words[center_start:(center_end+1)])

        matches.discard(word)
    else:
        matches = "No context for input word was found"
    return matches

## Using above functions

In [6]:
text3 = "a litte mouse was being chased by a cat, who we bought yesterday who was grey. there was a wolf who was running."

In [7]:
print("OUTPUT:\n\n" + str(approximate_similar(text3, 'cat', 6, 2)))

OUTPUT:

{'a', 'chased', 'wolf', 'was', '.', 'there', 'by'}


In [8]:
f = open('./approximate similar with context.txt')

f = f.readlines()
count  = 0
for i in f:
    f[count] = f[count].strip()
    count+=1

In [16]:
f

['there is not was a cat was running after a rat',
 'there was tiger after rat',
 'not a was there lion after running rat a',
 'fox after running a rat',
 'not there elephant']

In [9]:
str_ = ''
for i in range(0, len(f)):
    str_ += f[i]
    str_ += ' '

In [10]:
str_

'there is not was a cat was running after a rat there was tiger after rat not a was there lion after running rat a fox after running a rat not there elephant '

In [15]:
print("OUTPUT:\n")
print(approximate_similar(str_, 'cat', 6, 2))

OUTPUT:

{'a', 'after', 'fox', 'running', 'was', 'rat', 'tiger', 'is', 'not', 'there'}
